In [3]:
import numpy as np

vetices = np.load('/home/lirz6/PROGRAM/AE_LSTM/data/vertices.npy', allow_pickle=True)

tri = np.load('/home/lirz6/PROGRAM/AE_LSTM/data/triangles.npy')

The shape of `triangles.npy` is (334139, 3) and the shape of `vertices.npy` is (2, 201640).

This means:
*   There are 334,139 triangles.
*   Each triangle is defined by 3 vertex indices.
*   There are 201,640 vertices.
*   For each vertex, there are 2 coordinates (presumably longitude and latitude).

However, the `vertices` array shape `(2, 201640)` is a bit unusual for typical usage with Matplotlib's `tripcolor` or similar functions, which usually expect vertices as `(num_vertices, num_dimensions_per_vertex)`, e.g., `(201640, 2)`.

We should load and inspect `vertices.npy` to confirm its structure and transpose it if necessary before using it for plotting. The salinity data (or any data to be visualized on the mesh) should have a value for each of these 201,640 vertices.